In [1]:
# Import dependencies
import pandas as pd
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read data
players = pd.read_csv('../Resources/players.csv')
plays = pd.read_csv('../Resources/plays.csv')

week1 = pd.read_csv('../Resources/week1.csv')
week2 = pd.read_csv('../Resources/week2.csv')
week3 = pd.read_csv('../Resources/week3.csv')
week4 = pd.read_csv('../Resources/week4.csv')
week5 = pd.read_csv('../Resources/week5.csv')
week6 = pd.read_csv('../Resources/week6.csv')
week7 = pd.read_csv('../Resources/week7.csv')
week8 = pd.read_csv('../Resources/week8.csv')

tracking = week1.append([
    week2,
    week3,
    week4,
    week5,
    week6,
    week7,
    week8,
])

In [3]:
# Find tracking possession
player_tracking = tracking[tracking['team'] != 'football']
player_tracking = player_tracking.merge(plays[['gameId', 'playId', 'possessionTeam']])
player_tracking['possession'] = player_tracking['team'] == player_tracking['possessionTeam']

In [4]:
# Merge in position
player_tracking = player_tracking.copy().merge(players[['nflId', 'officialPosition']])

In [5]:
# Slice for QB 
qb_tracking = player_tracking[player_tracking['officialPosition'] == 'QB']

In [8]:
# Get nearest player
trouble_df = pd.DataFrame(columns=['gameId', 'playId', 'frameId', 'nflId', 'oppId'])
for i, r in qb_tracking.iterrows():
    # Check if play trouble moment is defined
    opp_df[
        (opp_df['gameId'] == r['gameId']) & \
        (opp_df['playId'] == r['playId'])
    ]
    if len(opp_df) == 0:
        # Get opposition data
        if r['possession']:
            players = player_tracking.loc[
                (player_tracking['gameId'] == r['gameId']) & \
                (player_tracking['playId'] == r['playId']) & \
                (player_tracking['frameId'] == r['frameId'])
            ]
        else:
            players = player_tracking.loc[
                (player_tracking['gameId'] == r['gameId']) & \
                (player_tracking['playId'] == r['playId']) & \
                (player_tracking['frameId'] == r['frameId'])
            ]

        # Find distance to each opposing player
        players['x_diff'] = r['x'] - players['x']
        players['x_diff'] = players['x_diff']**2

        players['y_diff'] = r['y'] - players['y']
        players['y_diff'] = players['y_diff']**2

        players['diff_sum'] = players['x_diff'] = players['y_diff']
        players['distance_opp'] = players['diff_sum'].apply(lambda x: math.sqrt(x))

        # Append nearest player
        players.sort_values(by=['distance_opp'])
        if not players['possession'].iloc[0]:
            opp_df = opp_df.append(pd.DataFrame({
                'gameId': players['gameId'].iloc[0],
                'playId': players['playId'].iloc[0],
                'frameId': players['frameId'].iloc[0],
                'troubleOppId': players['nflId'].iloc[0],
            }, index=[0]))
trouble_df

,gameId,playId,frameId,nflId,oppId,oppPossession
0,2021090900,137,1,43424.0,35441.0,False


In [9]:
# Slice for football data
football_tracking = tracking.copy().loc[tracking['team'] == 'football']
football_tracking.drop(['nflId', 'jerseyNumber', 'team', 'o', 'dir'], axis=1, inplace=True)

In [12]:
# Join in first trouble frame
football_trouble_tracking = football_tracking.merge(trouble_df[['gameId', 'playId', 'frameId', 'troubleOppId']], how='left')
football_trouble_tracking['firstTroubleFrame'] = football_trouble_tracking['troubleOppId'].notna()

In [ ]:
football_trouble_tracking.to_csv('football_trouble_event')

In [13]:
# Get trouble event frequency
football_trouble_tracking['firstTroubleFrame'].mean()

2.7663588631371616e-06